In [6]:
# 간단한 Transformer 기반 LLM 모델을 구축하는 파이썬 코드입니다.
# 이 코드는 GPT와 같은 언어 모델의 기본적인 아키텍처를 보여줍니다.

In [7]:
# 코드 설명
# Multi-Head Attention:
# MultiHeadAttention 클래스는 GPT와 같은 모델의 핵심 요소입니다. 여러 개의 어텐션 헤드를 이용해 병렬로 어텐션을 수행한 후, 이를 결합하여 최종 출력을 만듭니다.

# Decoder Layer:
# GPT는 디코더 기반 모델이므로, DecoderLayer 클래스는 Multi-Head Attention과 Feed Forward Network로 구성된 하나의 디코더 층입니다.
# 이 층에서는 Layer Normalization과 Residual 연결이 적용됩니다.

# GPT 모델:

# GPTModel 클래스는 여러 개의 디코더 층을 쌓아 구성한 전체 모델입니다. 각 층에서 입력 시퀀스가 처리됩니다.
# 이 모델은 **임베딩 층(Embedding Layer)**와 Positional Encoding을 사용하여 입력 시퀀스를 벡터로 변환하고, 이를 디코더에 전달합니다.

# Positional Encoding:

# Transformer는 시퀀스의 순서를 모델에 전달하기 위해 Positional Encoding을 추가합니다. 이는 각 위치 정보를 사인(sin)과 코사인(cos) 함수로 표현하여, 위치 정보를 학습할 수 있도록 합니다.

# GPT 구조의 특징
# Unidirectional: GPT는 한 방향성을 가지며, 이전 시점의 정보만을 사용해 다음 시점을 예측합니다. 이는 Masked Self-Attention을 통해 구현됩니다.
# 디코더만 사용: GPT는 전체 구조에서 디코더 부분만 사용하며, 언어 생성 작업에 특화되어 있습니다.

# 종합

In [8]:
!pip show tensorflow
!python --version

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras
Python 3.10.12


In [9]:
!python --version


Python 3.10.12


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, LayerNormalization, Dropout
from tensorflow.keras.models import Model

In [11]:
# Scaled Dot-Product Attention
def scaled_dot_product_attention(q, k, v, mask=None):
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

In [12]:



# Multi-Head Attention
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)
        return output, attention_weights

# Feed Forward Network
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        Dense(dff, activation='relu'),
        Dense(d_model)
    ])

# Transformer Decoder Layer (GPT 구조와 유사)
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

# GPT 모델 (Transformer 기반의 LLM)
class GPTModel(Model):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, max_pos_encoding, rate=0.1):
        super(GPTModel, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(max_pos_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = Dropout(rate)

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.dec_layers[i](x, training=training, mask=mask)

        return x

# Positional Encoding
import numpy as np

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)

    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])

    pos_encoding = np.concatenate([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

In [13]:



# 샘플 모델 테스트
sample_gpt = GPTModel(num_layers=2, d_model=512, num_heads=8, dff=2048, target_vocab_size=8000, max_pos_encoding=5000)

temp_input = tf.random.uniform((64, 20), dtype=tf.int64, minval=0, maxval=200)
output = sample_gpt(temp_input, training=False, mask=None)
print(output.shape)  # (batch_size, input_seq_len, d_model)


(64, 20, 512)


In [14]:
output

<tf.Tensor: shape=(64, 20, 512), dtype=float32, numpy=
array([[[ 0.52559865, -1.2146963 , -0.04999537, ..., -1.0526516 ,
          0.6155404 ,  0.5563539 ],
        [-0.20539647, -1.1764107 ,  1.0369886 , ..., -0.75341135,
          1.7446352 ,  0.57262176],
        [ 0.55239815, -0.303398  , -0.00522766, ..., -1.3012663 ,
          2.4997525 ,  0.22025205],
        ...,
        [-0.6086065 , -1.823865  , -0.32166487, ..., -1.5583954 ,
          1.470501  ,  1.3339434 ],
        [-0.4292233 , -1.88939   , -1.5638807 , ..., -1.2856418 ,
          2.2884107 ,  1.6459411 ],
        [-0.32540733, -2.4481504 , -1.4420972 , ..., -0.7427204 ,
          1.0035946 ,  1.562907  ]],

       [[-0.60038465, -1.2879398 , -0.8882612 , ..., -0.42889184,
          1.5921547 ,  0.74801326],
        [ 0.9600346 ,  0.7056459 , -0.24339516, ..., -0.23028536,
          1.1671729 ,  0.94767964],
        [ 0.57203394,  0.60987866,  1.1439922 , ..., -0.30341375,
          0.6386689 ,  1.5090677 ],
        ...,